# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features_600m.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(4614, 15051)


1800m_dft_dom_cycle_ddt_lag29        239
1800m_dft_dom_cycle_dt_lag29         236
1800m_dft_dom_cycle_ddt_lag28        236
1800m_dft_dom_cycle_dt_lag28         233
1800m_dft_dom_cycle_lag29            233
                                    ... 
600m_conv_18                           0
600m_conv_19                           0
600m_conv_20                           0
600m_conv_21                           0
600m_td_sequential_buy_aggressive      0
Length: 15051, dtype: int64

In [2]:
import numpy as np
from jesse import utils

label = np.load("data/label_side.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label["side_label"] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-02 21:00:00,6944.95,7848.9,8014.91,6863.44,717060.163,-1.0


In [4]:
BAD_FRONT = 239

side_features = df_features.iloc[BAD_FRONT:]
side_label = df_label.iloc[BAD_FRONT:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

(4375, 15051)
(4375,)


/var/folders/2g/md1t2j7s6hjdjc5jlvkmn7_c0000gn/T/ipykernel_83729/2066660060.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


1800m_adx_7                  0
200m_adx_7_lag2              0
600m_williams_r_ddt_lag22    0
600m_williams_r_ddt_lag23    0
600m_williams_r_ddt_lag24    0
                            ..
600m_adx_7_lag2              0
600m_adx_7_lag3              0
600m_adx_7_lag4              0
600m_adx_7_lag5              0
200m_williams_r_ddt_lag29    0
Length: 15051, dtype: int64

In [5]:
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector(max_features=2500)
selector.fit(side_features, side_label)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: 600m_vwap_dt (最大重要性: 0.0296)
➤ 计算特征冗余度...
➤ 总计选择2500个特征 (已选择1个，还需选择2499个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2499/2499 [04:58<00:00,  8.38特征/s]



✅ 特征选择完成：从15051个特征中选择了2500个，舍弃了12551个
✅ 选择的特征: ['600m_vwap_dt', '200m_dual_diff_ddt_lag11', '1800m_bekker_parkinson_vol', '1800m_adaptive_cci_ddt_lag23', '1800m_ehlers_early_onset_trend_ddt', '600m_williams_r_lag1', '600m_norm_on_balance_volume_dt', '200m_phase_accumulation_ddt_lag4', '600m_evenbetter_sinewave_short_dt', '600m_cmma', '1800m_acc_swing_index_lag27', '600m_williams_r_dt', '200m_stc_dt_lag1', '1800m_hurst_coef_slow_lag27', '600m_fisher_dt', '600m_adx_7_ddt', '600m_williams_r', '200m_cmma', '600m_dft_dom_cycle', '1800m_ehlers_early_onset_trend_ddt_lag16', '600m_stc_dt_lag1', '600m_acc_swing_index_dt', '600m_adx_14_dt', '600m_fisher', '200m_vwap_dt', '600m_stc_dt', '200m_acc_swing_index_lag2', '1800m_reactivity', '600m_natr_dt', '600m_williams_r_lag2', '600m_conv_0', '200m_stc_ddt', '200m_trendflex', '600m_fti_best_period_lag2', '600m_evenbetter_sinewave_short_ddt_lag23', '200m_stc_dt', '600m_cmma_dt', '600m_voss', '1800m_voss', '200m_fti_lag11', '1800m_williams_r', '1800m_

In [8]:
side_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
side_res

600m_vwap_dt                 0.029640
600m_williams_r_lag1         0.023716
600m_williams_r              0.021140
600m_cmma                    0.020380
600m_fisher_dt               0.016786
                               ...   
600m_aroon_diff_lag23        0.000000
600m_aroon_diff_lag24        0.000000
600m_aroon_diff_lag25        0.000000
600m_aroon_diff_lag26        0.000000
200m_williams_r_ddt_lag29    0.000000
Length: 15051, dtype: float64

In [9]:
import json

with open(f"data/side_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(side_res.to_dict(), f, indent=4)

In [10]:
import json

import pandas as pd

with open("data/side_features_RFCQSelector.json", "r") as f:
    side_res = pd.Series(json.load(f))

In [11]:
from strategies.BinanceMLV2.config import (
    DOLLAR_BAR_SHORT_TERM,
    DOLLAR_BAR_MID_TERM,
    DOLLAR_BAR_LONG_TERM,
)

feature_info = {
    "side": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MID_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
    },
    "meta": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MID_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
        "model_res": [],
    },
}

for k, v in side_res[side_res > 0].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["side"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MID_TERM}_"):
        feature_info["side"][DOLLAR_BAR_MID_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["side"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        raise ValueError(f"Unknown feature: {k}")

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MID_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MID_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("strategies/BinanceMLV2/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from strategies.BinanceMLV2.config import SIDE_ALL

df_features = pd.read_parquet("data/features_600m.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = lgb.Booster(model_file="strategies/BinanceMLV2/model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob

assert df_features.shape[0] == meta_label.shape[0]
assert "side_model_res" in df_features.columns

df_features.isna().sum(axis=0).sort_values(ascending=False)

1800m_dft_dom_cycle_ddt_lag29    239
1800m_dft_dom_cycle_dt_lag29     236
1800m_dft_dom_cycle_ddt_lag28    236
1800m_dft_dom_cycle_dt_lag28     233
1800m_dft_dom_cycle_lag29        233
                                ... 
600m_comb_spectrum_pwr_14          0
600m_comb_spectrum_pwr_13          0
600m_comb_spectrum_pwr_11          0
600m_comb_spectrum_pwr_10          0
side_model_res                     0
Length: 15052, dtype: int64

In [2]:
BAD_FRONT = 239

meta_features = df_features[BAD_FRONT:]
meta_label = meta_label[BAD_FRONT:]

meta_label = pd.Series(meta_label[:, 6].astype(int), index=meta_features.index)

# meta_features = meta_features[meta_label["ret"].notna()]
# meta_label = meta_label[meta_label["ret"].notna()]["bin"]

print(meta_features.shape)
print(meta_label.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(4375, 15052)
(4375,)


1800m_adx_7                  0
200m_adx_7_lag3              0
600m_williams_r_ddt_lag23    0
600m_williams_r_ddt_lag24    0
600m_williams_r_ddt_lag25    0
                            ..
600m_adx_7_lag2              0
600m_adx_7_lag3              0
600m_adx_7_lag4              0
600m_adx_7_lag5              0
side_model_res               0
Length: 15052, dtype: int64

In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector(max_features=2500)
selector.fit(meta_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: 200m_adx_14_dt (最大重要性: 0.0300)
➤ 计算特征冗余度...
➤ 总计选择2500个特征 (已选择1个，还需选择2499个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2499/2499 [05:11<00:00,  8.01特征/s]



✅ 特征选择完成：从15052个特征中选择了2500个，舍弃了12552个
✅ 选择的特征: ['200m_adx_14_dt', '1800m_dft_dom_cycle_lag24', '200m_mod_rsi_ddt_lag10', '1800m_chaiken_money_flow_lag17', '600m_trendflex_lag24', '200m_hurst_coef_slow_ddt_lag5', '1800m_pfe_ddt_lag23', '600m_cmma_lag7', '600m_dft_dom_cycle_ddt_lag6', '600m_pfe_dt_lag23', '1800m_aroon_diff_ddt_lag25', '1800m_evenbetter_sinewave_short_dt_lag17', '200m_change_variance_ratio_ddt_lag26', '600m_adx_14_ddt', '1800m_entropy_for_jesse_dt_lag24', '1800m_kyle_lambda_lag14', '600m_ma_difference_dt', '200m_price_variance_ratio_dt_lag23', '1800m_ac_13', '1800m_pfe_dt_lag15', '600m_dual_diff_lag24', '200m_price_change_oscillator_ddt_lag11', '600m_hasbrouck_lambda_dt', '600m_hurst_coef_slow_ddt_lag9', '200m_corwin_schultz_estimator_ddt_lag17', '1800m_hasbrouck_lambda_dt_lag18', '600m_hurst_coef_fast_dt_lag3', '1800m_kyle_lambda_dt_lag25', '1800m_price_change_oscillator_dt_lag21', '1800m_kyle_lambda_dt_lag10', '1800m_evenbetter_sinewave_long_dt_lag29', '1800m_adx_14_dt

In [4]:
import json

with open(f"data/meta_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(meta_res.to_dict(), f, indent=4)

In [5]:
import json

import pandas as pd

with open("data/meta_features_RFCQSelector.json", "r") as f:
    meta_res = pd.Series(json.load(f))

In [6]:
meta_res.to_dict()["side_model_res"]

0.010000000000000002

In [7]:
from pathlib import Path
from strategies.BinanceMLV2.config import (
    DOLLAR_BAR_SHORT_TERM,
    DOLLAR_BAR_MID_TERM,
    DOLLAR_BAR_LONG_TERM,
)

feature_info = Path("strategies/BinanceMLV2/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = []
feature_info["meta"][DOLLAR_BAR_MID_TERM] = []
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = []
feature_info["meta"]["model_res"] = []

for k, v in meta_res[meta_res > 0].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MID_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_MID_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        feature_info["meta"]["model_res"].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MID_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MID_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MID_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("strategies/BinanceMLV2/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)